In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame

# 1. 데이터 만들기 

In [233]:
# 데이터 불러오기
def load_all():
    train_activity = pd.read_csv('./train_activity.csv')
    train_combat = pd.read_csv('./train_combat.csv')
    train_payment = pd.read_csv('./train_payment.csv')
    train_trade = pd.read_csv('./train_trade.csv')
    train_pledge = pd.read_csv('./train_pledge.csv')
    train_label = pd.read_csv('./train_label_add.csv')
    return train_activity, train_combat, train_payment, train_trade, train_pledge, train_label

In [234]:
train_activity, train_combat, train_payment, train_trade, train_pledge, train_label = load_all()

In [236]:
# 1단계: char_id를 압축으로 인한 정보 손실 보강할 변수 추가

# 2단계: 모든 데이터를 acc_id, day 기준으로 합치기 (char_id는 압축)

# 2-1) 압축하기
activity = train_activity.groupby(['acc_id','day']).sum().reset_index().drop('char_id',axis=1)
combat = train_combat.groupby(['acc_id','day']).sum().reset_index().drop('char_id',axis=1)
payment = train_payment.groupby(['acc_id','day']).sum().reset_index()
pledge = train_pledge.groupby(['acc_id','day']).sum().reset_index().drop('char_id',axis=1)

In [237]:
#trade

# 주는 유저
train_trade_source = train_trade[['day', 'time', 'type', 'server', 'source_acc_id', 
                            'source_char_id', 'item_type', 'item_amount','item_price']]
# 받는 유저
train_trade_target = train_trade[['day', 'time', 'type', 'server', 'target_acc_id', 
                            'target_char_id', 'item_type', 'item_amount','item_price']]

train_trade_source = train_trade_source.rename(columns = {'source_acc_id': 'acc_id', 'source_char_id':'char_id',
                                                         'item_amount': 'item_amount_s', 'item_price': 'item_price_s'})
train_trade_target = train_trade_target.rename(columns = {'target_acc_id': 'acc_id', 'target_char_id':'char_id',
                                                         'item_amount': 'item_amount_t', 'item_price': 'item_price_t'})

# '시간' 컬럼을 그룹별로 합침(time, item_type, server 제거됨  )
train_trade_source = train_trade_source.groupby(['day','acc_id','char_id']).sum()
train_trade_source = train_trade_source.reset_index()

train_trade_target = train_trade_target.groupby(['day','acc_id','char_id']).sum()
train_trade_target = train_trade_target.reset_index()
train_trade_target['item_price_t'] = train_trade_target['item_price_t']*(-1)

standard = activity[['acc_id','day','playtime']]
source = pd.merge(standard, train_trade_source , on=['day','acc_id'], how = 'outer').dropna(subset = ['playtime']).drop(['char_id','playtime'],axis=1)
target = pd.merge(standard, train_trade_target , on=['day','acc_id'], how = 'outer').dropna(subset = ['playtime']).drop(['char_id','playtime'],axis=1)


In [238]:
trade = pd.merge(source, target , on=['day','acc_id'], how = 'outer')

In [239]:
trade = trade.groupby(['acc_id','day']).sum().reset_index()

In [240]:
# 데이터 합치기
data_ac = pd.merge(activity, combat, on = ['acc_id','day'])

# 현금 결제만 하고 플레이는 안 한 경우가 있어서, 인스턴스가 945628 -> 946632로 증가 (결측값을 0으로 채움)
data_acp = pd.merge(data_ac, payment, on = ['acc_id','day'], how='outer').fillna(0) 

data_acpl = pd.merge(data_acp, pledge, on = ['acc_id','day'], how='outer').fillna(0)
data_acplt = pd.merge(data_acpl, trade, on = ['acc_id','day'], how='outer').dropna(subset = ['playtime']).fillna(0)

In [241]:
data_acplt.shape

(946632, 42)

In [242]:
# 28*40000으로 데이터프레임 만들기

# 실제 접속일  데이터
real_day = data_acplt[['acc_id','day', 'playtime', 'npc_kill', 'solo_exp', 'party_exp',
       'quest_exp', 'rich_monster', 'death', 'revive', 'exp_recovery',
       'fishing', 'private_shop', 'game_money_change', 'enchant_count',
       'class', 'level', 'pledge_cnt', 'random_attacker_cnt_x',
       'random_defender_cnt_x', 'temp_cnt_x', 'same_pledge_cnt_x', 'etc_cnt_x',
       'num_opponent', 'amount_spent','play_char_cnt',
       'combat_char_cnt', 'pledge_combat_cnt', 'random_attacker_cnt_y',
       'random_defender_cnt_y', 'same_pledge_cnt_y', 'temp_cnt_y', 'etc_cnt_y',
       'combat_play_time', 'non_combat_play_time', 'item_amount_s',
       'item_price_s', 'item_amount_t', 'item_price_t']]
real_day = real_day.groupby(['acc_id','day']).sum().reset_index()

# 전체 일자 데이터 (40000*28)
day = np.zeros(shape=(40000*28,2))

# acc_id 40000개 추출
acc_id = train_activity[['acc_id','playtime']]
acc_id = acc_id.groupby('acc_id').sum().reset_index()
acc_id = acc_id['acc_id']
acc_id = acc_id.values

# day에 acc_id와 1~28일 입력 후 데이터프레임으로 변환
for i in range(0,40000):
    for j in range(0,28):
        n = 28*i+j
        day[n][0] = acc_id[i]
        day[n][1] = j+1
day_df = DataFrame(day, columns=['acc_id','day'])

# 만든 데이터프레임에 실제 접속일 데이터 merge
stop_count_df = pd.merge(day_df, real_day,  on=['acc_id','day'],how='outer').fillna(0)
# 접속하지 않은 날은 0, 접속일은 1로 표시
#stop_count_df['login'] = np.where(stop_count_df['playtime'].isnull(), 0, 1)
#stop_count_df['stop'] = 0
#stop_count_df['day'] = np.where(stop_count_df['playtime'].isnull(), 0, stop_count_df['day'])

In [243]:
# shape=(40000,28,39)인 데이터 틀 만듦
day = np.zeros(shape=(40000,28,39))
for i in range(0,40000):
    for j in range(0,28):
        day[i][j][0] = acc_id[i]
        day[i][j][1] = j+1

In [244]:
# 40000개의 3차원 인덱스에 acc_id별로 28*39 넣기
for i in range(0,40000):
    acc = acc_id[i]
    day[i] = stop_count_df[stop_count_df['acc_id']==acc]

In [215]:
day.shape

(40000, 28, 39)

In [235]:
# label 만들기
# 모든 데이터를 합친 data_acplt와 acc_id를 똑같이 정렬
train_label1 = train_label.sort_values('acc_id').reset_index()
# 클래스 레이블 별로 분리
survival_time = train_label1['survival_time'].values.reshape(40000,1)
amount_spent = train_label1['amount_spent'].values.reshape(40000,1)
secession = train_label1['secession'].values.reshape(40000,1)
total_spent = train_label1['total_spent'].values.reshape(40000,1)

# 2. LSTM 적용 

In [228]:
# 데이터 분리
train_x = day[:28000]
train_y = survival_time[:28000]
test_x = day[28000:]
test_y = survival_time[28000:]

In [230]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense


# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(28, 39)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(train_x, train_y, epochs=200, verbose=0)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


KeyboardInterrupt: 

In [ ]:
# demonstrate prediction
#x_input = array([[80, 85], [90, 95], [100, 105]])
#x_input = x_input.reshape((1, n_steps, n_features))
#yhat = model.predict(x_input, verbose=0)
#print(yhat)